# Data Prep

Preprocess and unify all data.

Before starting to train a ML model, we have to preprocess our data. In this case Sentinel-2 Level-2A imagery is used to generate composites by maximum NDVI across a period of two months. The resulting composites are augmented with indices, like NDVI and all timesteps are reduced into a single raster by deriving statistical parameters, like mean and variance.

The DEM image uploaded beforehand is downsampled to the same resolution as the Sentinel-2 composites by calculating various textile measures.

Then the resulting Sentinel-2 derived raster and DEM derived raster are stacked and a dimensionality reduction is performed. The reduced image can then be used for further processing.

## Define Parameters

In [1]:
# Import Earth Engine API and initialize it
import ee
ee.Initialize()

# Define data constants
SOURCE = 'COPERNICUS/S2_HARMONIZED'  # Define dataset source
REGION = ee.Geometry.Rectangle([12.6545, 47.9291, 12.6762, 47.9423])  # Define region in EPSG:4326

# Define processing constants
TIMESERIES_MIDDLE = '2016-01-01'  # Define middle of timeseries
TIMESERIES_DURATION = 365  # Define duration of timeseries in days
NUM_COMPOSITES = 12  # Define amount of composites in the timeseries
TEMPORAL_REDUCERS = [ee.Reducer.median(), ee.Reducer.variance()]  # Define temporal reducer

# Define quality measure for composites
def addQuality(image):
    quality_band = image.normalizedDifference(['B5', 'B4']).rename(['quality'])  # NDVI in this case
    return image.addBands(quality_band)

# Define export constants
FILENAME = 'NDVI_composite'  # Name of exported raster
FOLDER = 'Google Earth Engine'  # Name of export folder
SCALE = 10  # Size of pixel in meters
CRS = 'EPSG:32632'  # Coordinate reference system of exported raster
MAX_PIXELS = 1e7  # Maximum number of pixels when exporting

# Define map constants
VIS_PARAMS = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.2, 'gamma': 1}
LAYER_NAME = FILENAME

# Calculate Timeseries Windows for calculating Composites

In [2]:
from datetime import datetime, timedelta

def generate_timewindows(middle_date, num_windows, timeseries_duration=365.242):
    # Calculate the start date of the timeseries
    middle_date = datetime.strptime(middle_date, '%Y-%m-%d')
    current_start = middle_date - timedelta(days=timeseries_duration / 2)

    # Calculate the duration of each timewindow (in days)
    window_duration = timeseries_duration / num_windows
    
    # Initialize a list to store the timewindows as tuples
    timewindows = []
    for _ in range(num_windows):
        # Calculate the start and end dates of each timewindow
        start_date = current_start
        end_date = current_start + timedelta(days=window_duration)

        start_date = start_date.strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        timewindow = (start_date, end_date)
        
        # Append the timewindow as a tuple (start, end) to the list
        timewindows.append(timewindow)
        
        # Move the middle_date to the next timewindow
        current_start += timedelta(days=window_duration)
    
    return timewindows

timewindows = generate_timewindows(TIMESERIES_MIDDLE, NUM_COMPOSITES, TIMESERIES_DURATION)
print(timewindows)

[('2015-07-02', '2015-08-01'), ('2015-08-01', '2015-09-01'), ('2015-09-01', '2015-10-01'), ('2015-10-01', '2015-11-01'), ('2015-11-01', '2015-12-01'), ('2015-12-01', '2016-01-01'), ('2016-01-01', '2016-01-31'), ('2016-01-31', '2016-03-01'), ('2016-03-01', '2016-04-01'), ('2016-04-01', '2016-05-01'), ('2016-05-01', '2016-06-01'), ('2016-06-01', '2016-07-01')]


# Show Sentinel-2 Imagery

In [3]:
def maskS2clouds(image, cld_thresh=0.8, snw_thresh=0.8):
  # -----------------------------------------------------------------------
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  # -----------------------------------------------------------------------

  # Use MSK_CLDPRB and MSK_SNWPRB Level 2A bands with a threshold of 100
  # cld_prb = image.select('MSK_CLDPRB').divide(100)
  # snw_prb = image.select('MSK_SNWPRB').divide(100)
  # mask = cld_prb.lt(cld_thresh).And(snw_prb.lt(snw_thresh))

  return image.updateMask(mask).divide(10000)

def get_bands(image, print=False):
  bands = [band['id'] for band in image.getInfo()['bands']]

  if print:
    print(bands)

  return bands

def get_reducer_name(reducer, print=False):
  reducer_name = reducer.getInfo()['type'].split('.')[-1]

  if print:
    print(reducer_name)

  return reducer_name

In [4]:
# Define methods for calculating indices
def add_ndvi(image):
    ndvi_band = image.normalizedDifference(['B5', 'B4']).rename(['NDVI'])
    return image.addBands(ndvi_band)

def add_ndwi(image):
    ndwi_band = image.normalizedDifference(['B3', 'B5']).rename(['NDWI'])
    return image.addBands(ndwi_band)

add_indices = [add_ndvi, add_ndwi]

In [5]:
from IPython.display import Image

composites = []
for start, end in timewindows:
    # Define dataset filters
    filter_date = ee.Filter.date(start, end)  # inclusive start, exclusive end
    filter_region = ee.Filter.bounds(REGION)

    # Read dataset
    dataset = ee.ImageCollection(SOURCE).filter(filter_date).filter(filter_region).map(maskS2clouds)
    
    # Calculate indices
    for add_index in add_indices:
        dataset = dataset.map(add_index)

    # Create max NDVI pixel composite
    dataset = dataset.map(addQuality)  # Add quality band
    composite = dataset.qualityMosaic('quality')  # Choose max quality pixels

    # Remove quality band
    remaining_bands = composite.bandNames().getInfo()
    remaining_bands.remove('quality')
    composite = composite.select(remaining_bands)  # Remove quality band

    # Add to time series
    composites.append(composite)

# Create image collection
composites = ee.ImageCollection(composites)

# Apply remporal reducers to image collection
reduced_images = [composites.toBands()]  # TODO
# for temporal_reducer in TEMPORAL_REDUCERS:
#     reduced_images.append(composites.reduce(temporal_reducer))

# Stack images
stacked_images = ee.ImageCollection(reduced_images).toBands()
stacked_images = stacked_images.reproject(crs=CRS, scale=SCALE)

# Resample and clip to region
stacked_images = stacked_images.reproject(crs=CRS, scale=SCALE)
stacked_images = stacked_images.clip(REGION)

# Define RGB bands
rgb_bands = get_bands(stacked_images)[3:0:-1]

print('Stacked images:', stacked_images.bandNames().getInfo())

# Show image
Image(url=stacked_images.getThumbUrl({
    **VIS_PARAMS,
    'bands': rgb_bands,
    'dimensions': 500}))

Stacked images: ['0_0_B1', '0_0_B2', '0_0_B3', '0_0_B4', '0_0_B5', '0_0_B6', '0_0_B7', '0_0_B8', '0_0_B8A', '0_0_B9', '0_0_B10', '0_0_B11', '0_0_B12', '0_0_QA10', '0_0_QA20', '0_0_QA60', '0_0_NDVI', '0_0_NDWI', '0_1_B1', '0_1_B2', '0_1_B3', '0_1_B4', '0_1_B5', '0_1_B6', '0_1_B7', '0_1_B8', '0_1_B8A', '0_1_B9', '0_1_B10', '0_1_B11', '0_1_B12', '0_1_QA10', '0_1_QA20', '0_1_QA60', '0_1_NDVI', '0_1_NDWI', '0_2_B1', '0_2_B2', '0_2_B3', '0_2_B4', '0_2_B5', '0_2_B6', '0_2_B7', '0_2_B8', '0_2_B8A', '0_2_B9', '0_2_B10', '0_2_B11', '0_2_B12', '0_2_QA10', '0_2_QA20', '0_2_QA60', '0_2_NDVI', '0_2_NDWI', '0_3_B1', '0_3_B2', '0_3_B3', '0_3_B4', '0_3_B5', '0_3_B6', '0_3_B7', '0_3_B8', '0_3_B8A', '0_3_B9', '0_3_B10', '0_3_B11', '0_3_B12', '0_3_QA10', '0_3_QA20', '0_3_QA60', '0_3_NDVI', '0_3_NDWI', '0_4_B1', '0_4_B2', '0_4_B3', '0_4_B4', '0_4_B5', '0_4_B6', '0_4_B7', '0_4_B8', '0_4_B8A', '0_4_B9', '0_4_B10', '0_4_B11', '0_4_B12', '0_4_QA10', '0_4_QA20', '0_4_QA60', '0_4_NDVI', '0_4_NDWI', '0_5_B1', '0_

## Downsample DEM

In [7]:
from IPython.display import Image

# Load the DEM
dem = ee.Image('projects/leaf-type-mixture/assets/DEM')
band_name = dem.bandNames().getInfo()[0]

# Calculate textile measures
dem_mean = dem.reduceResolution(ee.Reducer.mean(), maxPixels=1024).rename(band_name + '_mean')
dem_variance = dem.reduceResolution(ee.Reducer.variance(), maxPixels=1024).rename(band_name + '_variance')
dem_glcm = dem.multiply(1000).toUint16().glcmTexture()  # naive approach to avoid memory issues

# Stack the DEM and its gradient
dem = dem.addBands([dem_mean, dem_variance, dem_glcm])

# Clip to the region of interest
dem = dem.reproject(crs=CRS, scale=SCALE)
dem = dem.clip(REGION)

# Print all bands
print(dem.bandNames().getInfo())

Image(url=dem.getThumbUrl({
    'min': -5,
    'max': 50,
    'bands': ['b1_shade', 'b1_diss', 'b1_prom'],
    'region': REGION,
    'dimensions': 500}))

['b1', 'b1_mean', 'b1_variance', 'b1_asm', 'b1_contrast', 'b1_corr', 'b1_var', 'b1_idm', 'b1_savg', 'b1_svar', 'b1_sent', 'b1_ent', 'b1_dvar', 'b1_dent', 'b1_imcorr1', 'b1_imcorr2', 'b1_maxcorr', 'b1_diss', 'b1_inertia', 'b1_shade', 'b1_prom']


# Rasterize Plot

In [11]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Constants, zero for conifers, one for broadleafs
CONIFER = 0
BROADLEAF = 1

# Map leaf type to a number
leaf_type_dict = {'Abies alba': CONIFER,
                  'Acer campestre': BROADLEAF,
                  'Acer platanoides': BROADLEAF,
                  'Acer pseudoplatanus': BROADLEAF,   
                  'Aesculus hippocastanum': BROADLEAF,
                  'Alnus glutinose': BROADLEAF,
                  'Betula ': BROADLEAF,
                  'Carpinus betulus': BROADLEAF,
                  'Fagus sylvatica': BROADLEAF,
                  'Fraxinus excelsior': BROADLEAF,
                  'Juglans regia': BROADLEAF,
                  'Larix decidua': CONIFER,
                  'Picea abies': CONIFER,
                  'Pinus sylvestris': CONIFER,
                  'Populus ': BROADLEAF,
                  'Populus tremula': BROADLEAF,
                  'Prunus avium': BROADLEAF,
                  'Pseudotsuga menziesii': CONIFER,
                  'Quercus ': BROADLEAF,
                  'Quercus rubra': BROADLEAF,
                  'Salix ': BROADLEAF,
                  'Sorbus aria': BROADLEAF,
                  'Sorbus aucuparia': BROADLEAF,
                  'Sorbus torminalis': BROADLEAF,
                  'Thuja plicata': CONIFER,
                  'Tilia ': BROADLEAF,
                  'Ulmus glabra': BROADLEAF,
                  'Unidentified broadleaf': BROADLEAF,
                  'Unidentified conifer': CONIFER}

# Read in the plot data
df = gpd.read_file('../data/raw/Plot.gpkg').to_crs('EPSG:4326')

# Create Longitude, Latitude, and Broadleaf columns
df['Longitude'] = df['geometry'].x
df['Latitude'] = df['geometry'].y
df['Species'] = df['Latin'] + ' ' + df['Mnemonic']
df['Broadleaf'] = df['Species'].map(leaf_type_dict)

# Keep only the columns we need
df = df[['Longitude', 'Latitude', 'Broadleaf']]

# Convert pandas DataFrame to Earth Engine FeatureCollection
fc = ee.FeatureCollection([ee.Feature(ee.Geometry.Point([row['Longitude'], row['Latitude']]), {'Broadleaf': row['Broadleaf']}) for index, row in df.iterrows()])

# Rasterize the FeatureCollection using reduceToImage
plot = fc.reduceToImage(['Broadleaf'], ee.Reducer.mean())
plot = plot.reproject(crs=CRS, scale=SCALE)
plot = plot.clip(REGION)

# Color palette
viridis_cmap = plt.get_cmap('viridis')
colors = []
for i in range(256):
    r, g, b, a = (int(value * 255) for value in viridis_cmap(i))
    colors.append(f'#{r:02X}{g:02X}{b:02X}')

Image(url=plot.getThumbUrl({
    'min': 0,
    'max': 1,
    'palette': colors,
    'region': REGION,
    'dimensions': 500}))

## Save (& fuse DEM with Time Series)

In [9]:
import requests
import rasterio
from rasterio.io import MemoryFile
import numpy as np

def download_image(image, scale, crs):
    download_params = {
        'scale': scale,
        'crs': crs,
        'format': 'GEO_TIFF',
    }
    url = image.getDownloadURL(download_params)

    return requests.get(url)

def save_image_to_file(image, file_path, mask, bands):
    with MemoryFile(image) as memfile, MemoryFile(mask) as mask_memfile:
        with memfile.open() as dataset, mask_memfile.open() as mask_dataset:
            profile = dataset.profile
            with rasterio.open(file_path, 'w', **profile) as dst:
                raster = dataset.read()
                mask = mask_dataset.read()
                raster[mask == 0] = np.nan
                dst.write(raster)
                dst.descriptions = tuple(bands)

def save_image(image, file_path, scale, crs):
    image_response = download_image(image, scale, crs)
    mask_response = download_image(image.mask(), scale, crs)
    
    if image_response.status_code == mask_response.status_code == 200:
        save_image_to_file(image_response.content,
                           file_path,
                           mask=mask_response.content,
                           bands=image.bandNames().getInfo())
        print('Image downloaded and saved successfully!')
    else:
        print('Failed to download the image.')

fusion = stacked_images.addBands(dem)

# # Merge all masks with logical AND operation
# band_names = fusion.bandNames().getInfo()
# mask = fusion.select(band_names[0]).mask()
# for band_name in band_names[1:]:
#     mask = mask.And(fusion.select(band_name).mask())
# band_names = plot.bandNames().getInfo()
# for band_name in band_names:
#     mask = mask.And(plot.select(band_name).mask())

# # Apply the mask to both images
# fusion = fusion.updateMask(mask)
# plot = plot.updateMask(mask)

fusion = fusion.clip(REGION)
plot = plot.clip(REGION)

# save_image
save_image(fusion, '../data/interim/fusion.tif', scale=SCALE, crs=CRS)
save_image(plot, '../data/interim/plot.tif', scale=SCALE, crs=CRS)

Image downloaded and saved successfully!
Image downloaded and saved successfully!


# Image to Dict

In [10]:
# Convert the image to grayscale
gray = stacked_images.select(['0_B4_median', '0_B3_median', '0_B2_median']).reduce(ee.Reducer.mean())  # reduce three bands to one
gray = gray.toUint16()

# Calculate the GLCM
glcm = gray.glcmTexture(3)

# Select the texture measures to export
texture_measures = glcm.select(
    ['mean_contrast', 'mean_diss', 'mean_shade', 'mean_asm', 'mean_prom'])

# Reduce the texture measures by mean
reduced_texture_measures = texture_measures.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=REGION,
    scale=SCALE,
    crs=CRS)

# Convert to dictionary
reduced_texture_measures = reduced_texture_measures.getInfo()
reduced_texture_measures

EEException: Image.select: Pattern '0_B4_median' did not match any bands.

# Interactive Map

In [ ]:
import folium
import matplotlib.pyplot as plt

# Create a map.
lon, lat = REGION.centroid().getInfo()['coordinates']
# lat, lon = 45.77, 4.855
my_map = folium.Map(location=[lat, lon], zoom_start=15)

# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Add the stacked composites to the map.
vis_params = {**VIS_PARAMS, 'bands': ['0_B4_median', '0_B3_median', '0_B2_median']}
my_map.add_ee_layer(stacked_images, vis_params, 'Sentinel-2 Composite')

# Get colors
viridis_cmap = plt.get_cmap('viridis')
colors = []
for i in range(256):
    r, g, b, a = (int(value * 255) for value in viridis_cmap(i))
    colors.append(f'#{r:02X}{g:02X}{b:02X}')

# Add the plot to the map.
vis_params = {
    'min': 0,'max': 1,
    'palette': colors
}
my_map.add_ee_layer(plot, vis_params, 'Leaf Type Mixture')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

# Experiments

In [ ]:
import folium

# Create a map.
lat, lon = 47.9357, 12.66535
my_map = folium.Map(location=[lat, lon], zoom_start=15)

# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Add the stacked composites to the map.
dem = ee.Image('projects/leaf-type-mixture/assets/DEM')
my_map.add_ee_layer(dem.multiply(100).toUint16().entropy(ee.Kernel.square(4)), {'min': 0, 'max': 10}, 'DEM')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)